<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [ ]:
from esper.prelude import *
from transcript_utils import *
from IPython.display import display, clear_output
from ipywidgets import Text, HTML
import random

In [ ]:
dataset = SegmentTextDataset(video_list())

In [ ]:
initial_segments = pcache.get('initial_segments')

In [ ]:
all_segments = set(range(len(dataset)))

In [ ]:
likely_positive = set([dataset.segment_index(s['item_name'], s['segment']) for s in initial_segments])
likely_negative = all_segments - likely_positive

In [ ]:
N = 3

def ngrams(segment, n):
    return [' '.join(segment[i:i+n]) for i in range(0, len(segment)+1-n)]

def label_widget(likely_positive, likely_negative, lexicon=[]):
    lex_set = set([ngram for [ngram, _] in lexicon])

    labels = []
    i = 0
    
    pos_idx = list(likely_positive)
    neg_idx = list(likely_negative)
    random.shuffle(pos_idx)
    random.shuffle(neg_idx)

    transcript = HTML(dataset[pos_idx[0]]['segment'])
    box = Text(placeholder='y/n')
    def on_submit(text):
        nonlocal i
        label = 1 if text.value == 'y' else 0
        (cur_source, next_source) = (pos_idx, neg_idx) if i % 2 == 0 else (neg_idx, pos_idx)        
        labels.append((cur_source[i//2], label))
        i += 1
        transcript.value = dataset[next_source[i//2]]['segment']
        box.value = ''
    box.on_submit(on_submit)  

    display(transcript)
    display(box)
    
    return labels

In [ ]:
labels = label_widget(likely_positive, likely_negative)

In [ ]:
print('Labels: {}'.format(len(labels)))
pcache.set('labeled_segments', labels)

In [ ]:
labels